In [ ]:
import warnings
warnings.filterwarnings('ignore')

# data and plotting
import pandas as pd
import numpy as np
from plotnine import *

# preprocessing
from sklearn.preprocessing import StandardScaler #Z-score variables
from sklearn.model_selection import train_test_split
# metrics
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error, ConfusionMatrixDisplay, roc_auc_score, recall_score, precision_score

# models
from sklearn.svm import SVC



<center><img src="https://drive.google.com/uc?export=view&id=1mSJFRHbiydTf3EvIIJr_kgw6Tfz1EXnb" alt="hyperplane plot" width = "400" class="center"/> </center>

In the lecture, we learned about Maximal Margin Classifiers, and Support Vector Classifiers. Both use hyperplanes (aka "flat affine subspaces") that divide our data into two sections.

<center><img src="https://drive.google.com/uc?export=view&id=1zAWlFxIOJchpnJRF3DGiIpv3VdFfq-ci" alt="hyperplane plot" width = "800" class="center"/> </center>

But if the data are linearly separable, there are infinite hyperplanes...HOW DO WE CHOOSE?


<center><img src="https://drive.google.com/uc?export=view&id=1m5GZObdjAFCjMwldvfyfSJkZPABYZr9c" alt="hyperplane plot" width = "400" class="center"/> </center>

### Question
<img src="https://drive.google.com/uc?export=view&id=1ghyQPx1N8dmU3MV4TrANvqNhGwnLni72" alt="Q" width = "200" class="center"/>

Why is it beneficial to maximize the margin as a way to separate two groups with a hyper plane?

</br>
</br>
</br>
But Maximal Margin Classifiers have a problem...

### Question
<img src="https://drive.google.com/uc?export=view&id=1ghyQPx1N8dmU3MV4TrANvqNhGwnLni72" alt="Q" width = "200" class="center"/>

How to SVCs solve the major issue with MMCs?


<center><img src="https://drive.google.com/uc?export=view&id=13OZpzK4-BCqjVsquR7XClrvHf64cGGaC" alt="Q" width = "400" class="center"/> </center>




# Constrained Optimization (Lagrangians)

## Maximal Margin Classifier $\mathcal{L}$
For Maximal Margin Classifiers, we're trying to *maximize* the *minimum* distance between the closest point(s) and the dividing hyperpalne. In otherwords, we're maximizing the function:

$$\underbrace{\frac{1}{||\mathbf{w}||} \underset{n}{\min}\left [ t_n(\mathbf{w}^Tx_n + b) \right ]}_\text{minimum distance from a point to hyperplane}$$

which is equivalent to minimizing $^1$:

$$\frac{1}{2}||\mathbf{w}||^2$$

subject to the constraint that all data points are at least *1 margin width* away from the hyperplane (aka none of them are inside the margin).

$$t_n(\mathbf{w}^Tx_n + b) \geq 1 \hspace{0.25in} \underbrace{\forall n}_\text{for all data points}$$


We can shove this function + constraint into a Lagrangian:

$$\mathcal{L}(\mathbf{w},b,\mathbf{\alpha}) = \frac{1}{2} \left \| \mathbf{w} \right \|^2 - \sum_{n = 1}^N \alpha_n \left \{ t_n(w^Tx_n + b)-1 \right \}$$

and set the partial derivatives of $\mathbf{w}$,$b$,and $\mathbf{\alpha}$ to 0 to find the optimal position of the hyperplane (defined by the weights $\mathbf{w}$ and bias $b$).
</br>
</br>
</br>
</br>

## What the Lagrange multipliers can tell us
When we use the Lagrangian to solve for our weights $\mathbf{w}$, our bias $b$, and our Lagrange multipliers $\alpha_n$, something *very* interesting pops out:

$$a_n\left \{t_ny(x_n) - 1 \right \} = 0$$

This formula says that the *product* of a data point's Lagrange multiplier $a_n$ and the point's distance to the hyperplane minus 1 (in other words $t_ny(x_n) - 1$) **must be zero**. The product of two numbers can only be zero when one or both of them is 0. So either:

1. The distance between the point and the hyperplane is 1 (aka it's ON the margin) or
2. its Lagrange multiplier is 0

Remember, the Lagrange multiplier (sometimes called the "shaddow price" in econ) tells us the **impact a constraint has on the optima of a function**. For example, if you're maximizing your profit subject to a budget your business has, the Lagrange multiplier tells you how much more profit you could make, if you increased your budget a little. 

In the MMC case, this tells us that *only* support vectors influence the position of the hyperplane. All other points have a Lagrange multiplier of 0. For example, removing these points from the data set would not change where the hyperplane is located.

<center><img src="https://drive.google.com/uc?export=view&id=1m5GZObdjAFCjMwldvfyfSJkZPABYZr9c" alt="hyperplane plot" width = "400" class="center"/> </center>

### Question
<img src="https://drive.google.com/uc?export=view&id=1ghyQPx1N8dmU3MV4TrANvqNhGwnLni72" alt="Q" width = "200" class="center"/>

What would happen to the hyperplane if we removed one of the *support vectors*?

</br>
</br>
</br>
</br>


---
$^1$ (we arbitrarily set the distance between the closest data point(s) and the hyperplane $\underset{n}{\min}\left [ t_n(\mathbf{w}^Tx_n + b) \right ]$ to be 1 to make the math easier. Maximizing $\frac{1}{||\mathbf{w}||}$ is the same as minimizing $\frac{1}{2}||\mathbf{w}||^2$ )


# Constraint/Budget vs. Penalty Formulation

We can think of SVCs in two ways:

1. As a penalty on the slack variables ($\xi_i$)

$$ \text{arg min} \frac{1}{2}\left | \mathbf{w} \right |^2 + C \sum_{i=0}^N \xi_i$$

Where $C$ controls how strongly we penalize non-zero slack variables.

2. As a constraint on the sum of the slack variables ($\xi_i$)
$$ \text{arg min} \frac{1}{2}\left | \mathbf{w} \right |^2 \text{ subject to } \sum_{i=0}^N \xi_i \leq C_{budget}$$

(this is similar to how we can think of LASSO/Ridge as a penalty on the coefficients OR as a constraint/budget on how large our coefficients can be)

# Making New Predictions
In both Support Vector Classifiers and Maximal Margin classifiers, we classify a data point by multiplying its predictors by the weights ($\mathbf{w}$) and adding the bias ($b$) and checking whether the value is > 0 (positive case, $t_n = 1$), or < 0 (negative case, $t_n = -1$).

In math terms, we calculate $\mathbf{w} \cdot x_n + b$ and see if it is > 0 or < 0.

</br>
</br>
</br>
</br>
</br>
</br>

# SVC in sklearn
Let's build a Support Vector Classifier together using the `df_together` data.

In [ ]:
from sklearn.datasets import make_blobs

blobs = make_blobs(n_samples = 100, random_state= 1234, centers = 2)
df_together = pd.DataFrame(blobs[0])
df_together.columns = ["X1", "X2"]
df_together["y"] = blobs[1]


In [ ]:
# plot data


In [ ]:
# split and organize data


In [ ]:
# build empty model

# fit model

# assess performance


# Classwork

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/cmparlettpelleriti/CPSC392ParlettPelleriti/master/Data/penguins.csv")

# get rid of chinstrap penguins
df = df.loc[(df.species == "Adelie") |  (df.species == "Gentoo")]
df.head()

## Try it Out
Here's a simplified penguin dataset that only has Gentoo and Adelie Penguins (Chinstrap penguins have been removed).

- Using ggplot, make a scatterplot of the bill length and bill depth data, coloring the points by species ([review of ggplot](https://github.com/cmparlettpelleriti/CPSC392ParlettPelleriti/blob/master/Lectures/LectureNotebooks/04_VisualizationI.ipynb))
- Looking at the ggplot, imagine where YOU would draw a line separating these two groups

### Question
<img src="https://drive.google.com/uc?export=view&id=1ghyQPx1N8dmU3MV4TrANvqNhGwnLni72" alt="Q" width = "200"/>

Looking at the data, can you see any reason why a Support Vector Classifier might be more desireable here than a Maximal Margin Classifier? Why? 

Where would the MMC put the dividing hyperplane? Where could SVC put it?

In [ ]:
df = df.dropna()


# plot the ggplot
### YOUR CODE HERE ###


- Split your data into a train/test split (80/20)
- z-score your predictors
- Build an empty `SVC()` model with `kernel = "linear"` and `C = 0.1`.
- fit the model
- plot the model using the function `plot_hyperplane()` that I wrote below
- print out the train/test accuracies, and roc aucs. 

### Question
<img src="https://drive.google.com/uc?export=view&id=1ghyQPx1N8dmU3MV4TrANvqNhGwnLni72" alt="Q" width = "200"/>

If you change the hyperparameter `C` to be 0.01, or 1, what happens to the margin? How many support vectors (points surrounded in red) are there with the different values of C?

In [ ]:
### YOUR CODE HERE ###

# organize and split data

In [ ]:
### YOUR CODE HERE ###
# make column transformer to z score

# create empty model

# put steps in pipeline
# the next cell is expecting pipeline with steps named "pre" and "model"

pipe = Pipeline([("pre", _________), ("model", _______________)])
pipe.fit(X_train, y_train)

# print out performance metrics

In [ ]:
###################### DON'T CHANGE JUST RUN TO LOAD FUNCTION ######################
def plot_hyperplane(svm, X):
    weights = svm.named_steps["model"].coef_[0]
    bias = svm.named_steps["model"].intercept_[0]
    slope = -weights[0]/weights[1]
    intercept = -bias/weights[1]

    X[X.columns] = svm.named_steps["pre"].transform(X)

    margin = 1/np.sqrt(np.sum(weights**2))
    lower_inter = intercept - (np.sqrt(1 + slope**2) * margin)
    upper_inter = intercept + (np.sqrt(1 + slope**2) * margin)
    
    cols = X.columns
    sv_df = pd.DataFrame(svm.named_steps["model"].support_vectors_)
    sv_df.columns = cols
    nice_cols = [c.replace("_", " ").title() for c in cols]
    

    a = (ggplot(X, aes(x = cols[0], y = cols[1])) +
    geom_point() +
    geom_abline(slope = slope, intercept = intercept,
                color = "red", linetype = "solid", size = 1) + 
        geom_abline(slope = slope, intercept = lower_inter,
                color = "gray", linetype = "dotted") +
        geom_abline(slope = slope, intercept = upper_inter,
                color = "gray", linetype = "dotted") + 
        theme_minimal() + 
        geom_point(data = sv_df, color = "red", size = 4, shape = "o", alpha = 0.25 ) + 
        labs(x = nice_cols[0],
        y = nice_cols[1],
        title = "Hyperplane and Margins") ) 
    return(a)

In [ ]:
# Call the plot_hyperplane function with your model and X_train

plot_hyperplane(pipe, X_train)


### Question
<img src="https://drive.google.com/uc?export=view&id=1ghyQPx1N8dmU3MV4TrANvqNhGwnLni72" alt="Q" width = "200"/>

Try a few more values for C that are bigger than our biggest C and smaller than our smallest C, Which values of C lead you to have something that looks more like a Maximal Margin Classifier?

## Another SVM Example

This time, use [this](https://raw.githubusercontent.com/cmparlettpelleriti/CPSC392ParlettPelleriti/master/Data/iris.csv) dataset to build a support vector machine using `sepal_width` and `sepal_length` to predict whether an iris flower is a `setosa` (coded as `1`), or `virginica` (coded as `-1`).

We can pull the intercept (bias) and coefficients (weights) from the SVM using `model.coef_` and `model.intercept_` just like we did for Linear and Logistic Regression in CPSC 392!

- Drop missing data (if any)
- Split your data into a train/test split (80/20)
- z-score your predictors
- Build an empty `SVC()` model with `kernel = "linear"` and `C = 0.1`.
- fit the model
- print out the train/test accuracies, and roc aucs.


In [ ]:
### YOUR CODE HERE ###
iris = pd.read_csv("https://raw.githubusercontent.com/cmparlettpelleriti/CPSC392ParlettPelleriti/master/Data/iris.csv")
# get rid of chinstrap penguins

# grab only Versi and Set irises
iris = iris.loc[(iris.species == "versicolor") |  (iris.species == "setosa")]

# head
iris.head()


In [ ]:
### YOUR CODE HERE ###
# make column transformer to z score

# create empty model

# put steps in pipeline

# fit pipeline

# print out performance metrics



### Margin Width

The formula for the margin width is:

$$ \frac{2}{\lVert \mathbf{w} \rVert}$$

Where $\lVert \mathbf{w} \rVert$ refers to the L2 norm of the weights $\mathbf{w} = \begin{bmatrix} w_1 \\ w_2 \\ ... \\ w_N\end{bmatrix}$.

Remember the L2 norm is calculated by:

$$ L_2 \text{ norm} = \sqrt{\sum_{i=1}^N w_i^2}$$


Using this math, use `model.coef_` to grab the weights/coefficients and calculate the width of the margin for your model using python (Hint: take advantage of `numpy`'s many vectorized functions e.g. `np.sqrt()` or `np.linalg.norm()`)

In [ ]:
# Calculate the width of the margin
### YOUR CODE HERE ###



### Slack Variables
Remember that Support Vector Classifiers improve upon Maximal Margin Classifiers by introducing slack variables $\xi_i$ that allow data points to violate the margin or even be on the wrong side of the hyperplane. 

Using the model you built for the last question to calculate the slack variables for the data point $z$ which is a random sample from our training data. 

Slack variables are calculated as:

$$\xi_i = max(0, \left |t_i - y(x_i) \right |)$$

Where $y(x_i)$ is the value $\mathbf{w}*x_n + b$. 

- Use the `plot_hyperplane()` function I wrote to plot the hyperplane for the iris dataset. Then discuss with your group:

1. Which regions of the graph have $\mathbf{w}*x_n + b > 0$? Which regions have $\mathbf{w}*x_n + b < 0$?
2. Which regions have slack variables ($\xi_i$) that are 0?
3. Which regions have slack variables ($\xi_i$) that are between 0 and 1?
4. What would have to happen for a slack variable ($\xi_i$) to be < 1?

In [ ]:
### YOUR CODE HERE ###
plot_hyperplane(pipe, X_train)

## Yet Another SVC Example

Using [this](https://raw.githubusercontent.com/cmparlettpelleriti/CPSC393ParlettPelleriti/main/Data/svmcw.csv) dataset, plot the data using ggplot to make a scatterplot of `X1` and `X2`, colored by `y` (the group of each data point).
### Question
<img src="https://drive.google.com/uc?export=view&id=1ghyQPx1N8dmU3MV4TrANvqNhGwnLni72" alt="Q" width = "200"/>

Is an SVM *able* to do a good job on this dataset?


Then, build an Support Vector Classifier to try to classify the data. How does it do?

- Drop missing data (if any)
- Split your data into a train/test split (80/20)
- z-score your predictors
- Build an empty `SVC()` model with `kernel = "linear"` and `C = 0.001`.
- fit the model
- print out the train/test accuracies, and roc aucs.

In [ ]:
### YOUR CODE HERE ###



In [ ]:
# plot

### YOUR PLOT HERE ###


In [ ]:
# Do your train test split


In [ ]:
# make column transformer to z score

# create empty model

# put steps in pipeline

# fit pipeline

# print out performance metrics



